# Predict pollution

## Parameters

In [1]:
from pathlib import Path

BASE_DIR = Path('/Users/efraflores/Desktop/EF/Diplo/data/04/pollution')
FILE_NAME = 'contaminantes.csv'
EPOCHS = 22
BATCH_SIZE = 2

## Import

In [2]:
import pandas as pd

df = pd.read_csv(BASE_DIR.joinpath(FILE_NAME))
df.sample()

,id,fecha,nom_estac,clave_de_estacion,clave_parametro,nombre_de_parametro,unidades_parametros,valor,id_unidad,clave_de_unidad,nombre_de_unidad,alt,obs_estac,id_station,latitud,longitud,coordenadas
206384,206384,2020-03-18T19:00:00+00:00,Milpa Alta,MPA,O3,Ozono,1,77.0,1,ppb,partes por billón,2594,NaN,484090090104,19.1769,-98.990189,"19.1769,-98.990189"


## Transform

### Date variables

In [3]:
df['fecha'] = pd.to_datetime(df['fecha']).dt.tz_convert('America/Mexico_City')
df['date'] = df['fecha'].dt.date.astype(str)
df['hour'] = df['fecha'].dt.hour
df.sample()

,id,fecha,nom_estac,clave_de_estacion,clave_parametro,nombre_de_parametro,unidades_parametros,valor,id_unidad,clave_de_unidad,nombre_de_unidad,alt,obs_estac,id_station,latitud,longitud,coordenadas,date,hour
180228,180228,2020-12-02 06:00:00-06:00,Pedregal,PED,PM10,Partículas menores a 10 micrómetros,2,27.0,2,µg/m³,microgramos por metro cúbico,2326,NaN,484090100127,19.325146,-99.204136,"19.325146,-99.204136",2020-12-02,6


### Hour range

In [4]:
df['hour_range'] = pd.cut(df['hour'], bins=[-1, 3, 7, 11, 15, 19, 24]).map(lambda x: str(x.left+1).zfill(2)+' to '+str(x.right).zfill(2))
df['hour_range'].value_counts(1, dropna=False)

00 to 03    0.166667
04 to 07    0.166667
08 to 11    0.166667
12 to 15    0.166667
16 to 19    0.166667
20 to 24    0.166667
Name: hour_range, dtype: float64

### Which station and param?

In [5]:
station_param = []
for station in set(df['clave_de_estacion']):
    same_station = df[df['clave_de_estacion']==station].copy()
    for param in set(same_station['clave_parametro']):
        same_param = same_station[same_station['clave_parametro']==param].copy()
        tot_dates = pd.DataFrame(pd.date_range(start=same_param['fecha'].dt.date.min(), end=same_param['fecha'].dt.date.max()), columns=['date'])
        non_registered = tot_dates.astype(str).merge(same_param.groupby('date')['valor'].sum().reset_index(), how='left')['valor'].isnull()
        station_param.append((station, param, non_registered.sum(), non_registered.mean()))

In [6]:
spn = pd.DataFrame(station_param, columns=['station','param','null','%_null'])
spn['null'].describe()

count    215.0
mean     236.0
std        0.0
min      236.0
25%      236.0
50%      236.0
75%      236.0
max      236.0
Name: null, dtype: float64

### Choose station

In [7]:
del spn
df = df[df['clave_de_estacion']=='FAC'].reset_index(drop=True)
df.sample()

,id,fecha,nom_estac,clave_de_estacion,clave_parametro,nombre_de_parametro,unidades_parametros,valor,id_unidad,clave_de_unidad,nombre_de_unidad,alt,obs_estac,id_station,latitud,longitud,coordenadas,date,hour,hour_range
8153,292243,2020-03-27 20:00:00-06:00,FES Acatlán,FAC,O3,Ozono,1,13.0,1,ppb,partes por billón,2299,NaN,484150570109,19.482473,-99.243524,"19.482473,-99.243524",2020-03-27,20,20 to 24


### All dates

In [8]:
df = pd.DataFrame(pd.date_range(start=df['fecha'].dt.date.min(), end=df['fecha'].dt.date.max()), columns=['date']).astype(str).merge(df, how='left')
df.isnull().sum()

date                       0
id                       236
fecha                    236
nom_estac                236
clave_de_estacion        236
clave_parametro          236
nombre_de_parametro      236
unidades_parametros      236
valor                    808
id_unidad                236
clave_de_unidad          236
nombre_de_unidad         236
alt                      236
obs_estac              13340
id_station               236
latitud                  236
longitud                 236
coordenadas              236
hour                     236
hour_range               236
dtype: int64

### Missing dates

In [9]:
missing_dates = df.fillna({'hour':'00'}).pivot_table(index='date', columns='hour', values='valor', aggfunc='sum').melt(ignore_index=False).reset_index()
missing_dates.sort_values(['date','hour'], inplace=True)
missing_dates.isnull().sum()

date        0
hour        0
value    6055
dtype: int64

In [10]:
import cufflinks as cf
cf.go_offline()
missing_dates.set_index(['date','hour']).iplot()

In [11]:
missing_dates.groupby('date')['value'].sum().iplot()

### Expand

In [12]:
expanded = df.copy()
del df, missing_dates
expanded['valor'] = expanded['valor'].fillna(method='bfill')
expanded = expanded.pivot_table(index='date', columns=['clave_parametro','hour_range'], values='valor', aggfunc=['count','sum','mean','median','min','max'])
expanded.fillna(0, inplace=True)
expanded.columns = ['_'.join(x) for x in expanded.columns]
expanded.head()

,count_CO_00 to 03,count_CO_04 to 07,count_CO_08 to 11,count_CO_12 to 15,count_CO_16 to 19,count_CO_20 to 24,count_NO_00 to 03,count_NO_04 to 07,count_NO_08 to 11,count_NO_12 to 15,...,max_PM10_08 to 11,max_PM10_12 to 15,max_PM10_16 to 19,max_PM10_20 to 24,max_SO2_00 to 03,max_SO2_04 to 07,max_SO2_08 to 11,max_SO2_12 to 15,max_SO2_16 to 19,max_SO2_20 to 24
date,,,,,,,,,,,,,,,,,,,,,
2019-12-31,0,0,0,0,1,4,0,0,0,0,...,0.0,0.0,52.0,124.0,0.0,0.0,0.0,0.0,6.0,4.0
2020-01-01,4,4,4,4,4,4,4,4,4,4,...,46.0,26.0,20.0,14.0,5.0,4.0,2.0,2.0,20.0,13.0
2020-01-02,4,4,4,4,4,4,4,4,4,4,...,71.0,76.0,55.0,47.0,42.0,25.0,19.0,55.0,73.0,4.0
2020-01-03,4,4,4,4,3,0,4,4,4,4,...,37.0,102.0,47.0,0.0,3.0,5.0,2.0,2.0,2.0,0.0
2020-01-04,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Shifted

In [13]:
shifted = expanded.copy()
del expanded
for col in shifted.columns:
    for i in range(1,21):
        shifted[f'{str(i).zfill(2)}_{col}'] = shifted[col].shift(i)
shifted.dropna(inplace=True)
shifted.shape

(319, 4536)

## Model

### Preprocessing

#### f(X) = y

In [14]:
X = shifted.filter(regex='^\d')
y = shifted.filter(regex='^sum_O3').sum(axis=1).values
del shifted
X.shape

(319, 4320)

#### Train test split

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.77, shuffle=False)

#### PCA & Scaler

In [16]:
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

pipe = Pipeline(steps=[('pre_scaler', RobustScaler()), ('dim_red', PCA(0.9999)),('pos_scaler', MinMaxScaler())])
X_train = pipe.fit_transform(X_train)
X_test = pipe.transform(X_test)
X_train.shape

(245, 200)

#### Target scaler

In [17]:
mm_y = RobustScaler()
print(min(y_train),max(y_train))
y_train = mm_y.fit_transform(y_train.reshape((y_train.shape[0], -1)))
y_test = mm_y.transform(y_test.reshape((y_test.shape[0], -1)))
print(min(y_train),max(y_train))

0.0 1040.0
[0.] [2.48210024]


#### Reshape

In [18]:
X_train = X_train.reshape(X_train.shape[0], -1, X_train.shape[1])
X_test = X_test.reshape(X_test.shape[0], -1, X_test.shape[1])
X_train.shape

(245, 1, 200)

### Arquitecture

In [19]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

model = Sequential()
model.add(LSTM(10, input_shape=X_train.shape[1:], activation="tanh"))
model.add(Dense(1))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 10)                8440      
_________________________________________________________________
dense (Dense)                (None, 1)                 11        
Total params: 8,451
Trainable params: 8,451
Non-trainable params: 0
_________________________________________________________________


#### Callbacks

In [21]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

early_stopping = EarlyStopping(monitor='val_mae', patience=20)
checkpoint = ModelCheckpoint(BASE_DIR.joinpath('models','pollution_model_{val_mae:.3f}.h5'),
                             save_best_only=True,
                             save_weights_only=False,
                             monitor='val_mae')

#### Metrics

In [22]:
from tensorflow.keras import metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

kmetrics = [metrics.RootMeanSquaredError(name='rms'), metrics.MeanAbsoluteError(name='mae')]

### Training

#### Compile

In [23]:
model.compile(loss='mean_squared_error', optimizer='adam', metrics=kmetrics)

#### Fit

In [24]:
training_history = model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(X_test, y_test), callbacks=[checkpoint, early_stopping])

Epoch 1/22
123/123 [==============================] - 2s 5ms/step - loss: 0.5241 - rms: 0.7239 - mae: 0.6058 - val_loss: 0.2693 - val_rms: 0.5189 - val_mae: 0.4392
Epoch 2/22
123/123 [==============================] - 0s 1ms/step - loss: 0.5101 - rms: 0.7142 - mae: 0.6057 - val_loss: 0.2899 - val_rms: 0.5384 - val_mae: 0.4809
Epoch 3/22
123/123 [==============================] - 0s 1ms/step - loss: 0.4893 - rms: 0.6995 - mae: 0.6065 - val_loss: 0.2392 - val_rms: 0.4891 - val_mae: 0.3300
Epoch 4/22
123/123 [==============================] - 0s 1ms/step - loss: 0.4977 - rms: 0.7054 - mae: 0.5895 - val_loss: 0.2692 - val_rms: 0.5189 - val_mae: 0.4455
Epoch 5/22
123/123 [==============================] - 0s 2ms/step - loss: 0.4886 - rms: 0.6990 - mae: 0.5976 - val_loss: 0.2644 - val_rms: 0.5142 - val_mae: 0.4386
Epoch 6/22
123/123 [==============================] - 0s 1ms/step - loss: 0.4597 - rms: 0.6780 - mae: 0.5681 - val_loss: 0.2663 - val_rms: 0.5160 - val_mae: 0.4522
Epoch 7/22
123/1

#### Metrics

In [25]:
metrics = pd.DataFrame(data = zip(training_history.history["loss"], training_history.history["val_loss"], training_history.history["mae"], training_history.history["val_mae"]), columns=["loss", "val_loss", "mae", "val_mae"])
metrics.iplot()

### Predict

#### Preprocessing

In [26]:
prep = pipe.transform(X)
prep = prep.reshape((prep.shape[0],-1,prep.shape[-1]))
prep.shape

(319, 1, 200)

#### Prediction

In [27]:
from numpy import clip
pred = X.copy()
pred['real'] = y
pred['est'] = mm_y.inverse_transform(model.predict(prep))
pred['est'] = clip(pred['est'].values, 0, 1e3)
del prep

## Results

In [28]:
pred[['real','est']].iplot()